In [1]:
import numpy as np
import pandas as pd
import pickle
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats

import numpy

# Loading the data

## Specifying gesture

In [74]:
gesture_name = "click"
subjects = [f"Subject{i}" for i in range(1, 6)]

In [75]:
express = False

In [79]:
def get_data(gesture_name):
    subjects = [f"Subject{i}" for i in range(1, 6)]

    data = list()

    for subject in subjects:
        with open(f"data/randomized/{subject}/50cm/{gesture_name}", "rb") as file:
            temp = pickle.load(file)
            data += temp

        with open(f"data/randomized/{subject}/75cm/{gesture_name}", "rb") as file:
            temp = pickle.load(file)
            data += temp
    
        with open(f"data/randomized/{subject}/100cm/{gesture_name}", "rb") as file:
            temp = pickle.load(file)
            data += temp

    def normalize(z: float, max: float, min: float) -> float:
        norm = (z - min)/(max - min)
        round_norm = round(norm, 3)
        
        return round_norm

    def normalize_landmarks(landmarks: list) -> list:
        norm_landmarks = list()

        for landmark in landmarks:
            xs = [landmark[i][0] for i in range(len(landmark))]
            ys = [landmark[i][1] for i in range(len(landmark))]

            xmax = max(xs)
            ymax = max(ys)
            xmin = min(xs)
            ymin = min(ys)

            norm_landmark = list()

            for x, y in zip(xs, ys):
                norm = [normalize(z=x, max=xmax, min=xmin), normalize(z=y, max=ymax, min=ymin)]
                norm_landmark.append(norm)

            norm_landmarks.append(norm_landmark)

        return norm_landmarks

    norm_data = normalize_landmarks(landmarks=data)

    return norm_data

In [82]:
gesture_names = ["click", "closed", "down", "mouse_tracking", "negative_closed", "negative_mouse_tracking",
                 "negative_side", "negative_up", "side", "up"]

gestures_data = {id: get_data(gesture_name=gesture) for id, gesture in enumerate(gesture_names)}

In [85]:
df = pd.DataFrame(
    columns=["x médio", "y médio", "x std", "y std", "gesto"],
    index=[i for i in range(10*21)]
    )

In [92]:
i = 0
for gesture, norm_data in gestures_data.items():
    xs = dict({i: list() for i in range(21)})
    ys = dict({i: list() for i in range(21)})

    for landmarks in norm_data:
        for point, marks in enumerate(landmarks):
            xs[point].append(marks[0])
            ys[point].append(marks[1])
    
    for point in range(21):
        x_mean = np.mean(xs[point])
        y_mean = np.mean(ys[point])

        x_max = np.max(xs[point])
        y_max = np.max(ys[point])

        x_min = np.min(xs[point])
        y_min = np.min(ys[point])

        x_std = np.std(xs[point])
        y_std = np.std(ys[point])

        df.loc[i]["x médio"] = x_mean
        df.loc[i]["y médio"] = y_mean
        df.loc[i]["x std"] = x_std
        df.loc[i]["y std"] = y_std
        df.loc[i]["gesto"] = gesture
        
        i += 1

In [93]:
df

,x médio,y médio,x std,y std,gesto
0,0.272398,0.997778,0.070862,0.047088,0
1,0.515549,0.901551,0.062449,0.044404,0
2,0.697342,0.751391,0.048294,0.042961,0
3,0.85606,0.654296,0.022941,0.05534,0
4,0.99946,0.606276,0.011442,0.077569,0
...,...,...,...,...,...
205,0.756836,0.075653,0.158542,0.046731,9
206,0.926993,0.633002,0.097259,0.026673,9
207,0.904689,0.461867,0.071787,0.031344,9
208,0.876496,0.467324,0.10241,0.049927,9


In [60]:
for point in range(21):
    x_mean = np.mean(xs[point])
    y_mean = np.mean(ys[point])

    x_max = np.max(xs[point])
    y_max = np.max(ys[point])

    x_min = np.min(xs[point])
    y_min = np.min(ys[point])

    x_std = np.std(xs[point])
    y_std = np.std(ys[point])

    df.loc[point]["x médio"] = x_mean
    df.loc[point]["y médio"] = y_mean
    df.loc[point]["x std"] = x_std
    df.loc[point]["y std"] = y_std

In [61]:
df

,x médio,y médio,x std,y std
0,0.272398,0.997778,0.070862,0.047088
1,0.515549,0.901551,0.062449,0.044404
2,0.697342,0.751391,0.048294,0.042961
3,0.85606,0.654296,0.022941,0.05534
4,0.99946,0.606276,0.011442,0.077569
5,0.445547,0.491651,0.046967,0.019051
6,0.448711,0.294624,0.07695,0.019734
7,0.4369,0.173984,0.101034,0.031546
8,0.416604,0.06704,0.120657,0.042593
9,0.291169,0.497713,0.036529,0.01423


In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(2, 5)


fig.add_trace(go.Scatter(
    x=df["x médio"],
    y=df["y médio"],
    mode="markers",
    error_y=dict(
        type="data",
        array=df["y std"],
        color="red"
    ),    
    error_x=dict(
        type="data",
        array=df["x std"],
        color="red"
    ),
    marker=dict(color="blue", size=10),
    facet_col=
    )
)

fig.update_layout(
    xaxis = dict(autorange="reversed"),
    yaxis = dict(autorange="reversed"),
    yaxis_title="y médio [pixel normalizado]",
    xaxis_title="x médio [pixel normalizado]",
)

fig.show()

In [73]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df["x médio"],
    y=df["y médio"],
    mode="markers",
    error_y=dict(
        type="data",
        array=df["y std"],
        color="red"
    ),    
    error_x=dict(
        type="data",
        array=df["x std"],
        color="red"
    ),
    marker=dict(color="blue", size=10),
    facet_col=
    )
)

fig.update_layout(
    xaxis = dict(autorange="reversed"),
    yaxis = dict(autorange="reversed"),
    yaxis_title="y médio [pixel normalizado]",
    xaxis_title="x médio [pixel normalizado]",
)

fig.show()

In [56]:
import plotly.express as px
df = px.data.tips()
fig = px.scatter(df, x="total_bill", y="tip", color="smoker", facet_col="sex", facet_row="time")
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


# Using the Reference

In [9]:
with open("reference", "rb") as file:
    reference = pickle.load(file)

norm_reference = normalize_landmarks(landmarks=[reference])[0]
norm_reference

[[0.43, 1.0],
 [0.609, 0.927],
 [0.754, 0.808],
 [0.877, 0.706],
 [1.0, 0.65],
 [0.559, 0.497],
 [0.592, 0.299],
 [0.609, 0.175],
 [0.609, 0.056],
 [0.436, 0.486],
 [0.43, 0.266],
 [0.419, 0.124],
 [0.402, 0.0],
 [0.324, 0.531],
 [0.251, 0.339],
 [0.207, 0.215],
 [0.173, 0.102],
 [0.229, 0.621],
 [0.123, 0.492],
 [0.056, 0.412],
 [0.0, 0.328]]